## Context:

Today we are going to be working with the IMDB movie data set.  Our goal is to eventually create a linear regression model that will enable us to predict the box office gross of a movie based on characteristics of the movie.

Before we can start to model, we need to make sure our data is clean an in a usable format.  Therefore we will go through several steps of data cleaning. The code below is not a fully exhaustive list, but includes many of the process you will go through to clean data.  

In [1]:

import numpy as np 
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 100)

### Check Your Data … Quickly
The first thing you want to do when you get a new dataset, is to quickly to verify the contents with the .head() method.

In [1]:
df = pd.read_csv('movie_metadata.csv')
print(df.shape)
df.head()


NameError: name 'pd' is not defined

Now let’s quickly see the names and types of the columns. Most of the time you’re going get data that is not quite what you expected, such as dates which are actually strings and other oddities. But to check upfront.

In [3]:
# Get column names
column_names = df.columns
print(list(column_names))



['color', 'director_name', 'num_critic_for_reviews', 'duration', 'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name', 'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name', 'movie_title', 'num_voted_users', 'cast_total_facebook_likes', 'actor_3_name', 'facenumber_in_poster', 'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews', 'language', 'country', 'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes', 'imdb_score', 'aspect_ratio', 'movie_facebook_likes']


In [4]:
# Get column data types
df.dtypes

color                         object
director_name                 object
num_critic_for_reviews       float64
duration                     float64
director_facebook_likes      float64
actor_3_facebook_likes       float64
actor_2_name                  object
actor_1_facebook_likes       float64
gross                        float64
genres                        object
actor_1_name                  object
movie_title                   object
num_voted_users                int64
cast_total_facebook_likes      int64
actor_3_name                  object
facenumber_in_poster         float64
plot_keywords                 object
movie_imdb_link               object
num_user_for_reviews         float64
language                      object
country                       object
content_rating                object
budget                       float64
title_year                   float64
actor_2_facebook_likes       float64
imdb_score                   float64
aspect_ratio                 float64
m

## Convert a column to a different data type

The most common example of this is converting a string of number to an actual float or integer.  There are two ways you can achieve this.  

1. astype(float) method

`df['DataFrame Column'] = df['DataFrame Column'].astype(float)`
2.  to_numeric method

`df['DataFrame Column'] = pd.to_numeric(df['DataFrame Column'],errors='coerce')`

What is the difference in these two methods?

(1) For a column that contains numeric values stored as strings;

(2) For a column that contains both numeric and non-numeric values. By setting errors=’coerce’, you’ll transform the non-numeric values into NaN.


https://datatofish.com/convert-string-to-float-dataframe/

In [5]:
df['num_critic_for_reviews'] = df['num_critic_for_reviews'].astype(str, errors='ignore')

In [6]:
df['num_critic_for_reviews'][0]

'723.0'

In [7]:
 pd.to_numeric(df['num_critic_for_reviews'],errors='coerce', downcast='integer')

0       723.0
1       302.0
2       602.0
3       813.0
4         NaN
        ...  
5038      1.0
5039     43.0
5040     13.0
5041     14.0
5042     43.0
Name: num_critic_for_reviews, Length: 5043, dtype: float64

In [8]:
df['num_critic_for_reviews'] = pd.to_numeric(df['num_critic_for_reviews'],errors='coerce', downcast='integer')

In [9]:
df['title_year'][0]

2009.0

In [10]:
df['title_year']= pd.to_datetime(df['title_year'], format='%Y')
df['title_year'].head(10)

0   2009-01-01
1   2007-01-01
2   2015-01-01
3   2012-01-01
4          NaT
5   2012-01-01
6   2007-01-01
7   2010-01-01
8   2015-01-01
9   2009-01-01
Name: title_year, dtype: datetime64[ns]

In [12]:
df.

SyntaxError: invalid syntax (<ipython-input-12-38e9e33c87a8>, line 1)

### Drop Columns

If you do not plan on using some data in your analysis, feel free to drop those columns. 

In [13]:
print(df.columns)

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')


In [ ]:
df.drop(columns=['aspect_ratio', 'plot_keywords'], inplace=True)

In [ ]:
df.shape

In [ ]:
smaller_df=df[['gross','budget']]

## Investigate the data

In [ ]:
df.content_rating

In [ ]:
#look at the unique values for ratings
ratings = list(df['content_rating'].unique())
ratings

In [ ]:
df['content_rating'].value_counts()

There are many unique values that don't have a high count or don't make sense to the common user.  How should we handle these?

In [ ]:
#create a list of the ratings we want to group
unrated = ['Unrated','Approved', 'Not Rated', 'TV-MA', 'M', 'GP', 'Passed', np.nan, 'X', 'NC-17','TV-14', 'TV-PG', 'TV-G', 'TV-Y', 'TV-Y7']

In [ ]:
#create a list of the movie ratings we want to maintian
rated = [x for x in ratings if x not in unrated]

In [ ]:
rated

In [ ]:
#create a dictionary with keys of the 'unrated' values and the value being 'unrated'
unrated_dict = dict.fromkeys(unrated, 'unrated')

In [ ]:
unrated_dict

In [ ]:
#create a dictionary of the rated values
rated_dict  = dict(zip(rated, rated))

In [ ]:
rated_dict

In [ ]:
#combine those ditionaries into 1
ratings_map = {**rated_dict,**unrated_dict}
ratings_map

#### What does `**` do? 

It basically takes the dictionary passed through and unpacks it.  

https://medium.com/understand-the-python/understanding-the-asterisk-of-python-8b9daaa4a558

https://pynash.org/2013/03/13/unpacking/

In [ ]:
# use the pandas map function to change the content_rating column
df['rating'] = df['content_rating'].map(ratings_map)

In [ ]:
#compare the two columns
df[['rating', 'content_rating']].tail()

## Handling Missing Data:
    


In [ ]:
df.head()

In [ ]:
#creates a dataframe of booleans show where data is missing
df.isna().head()

In [ ]:
# Find the Percentage of rows missing data
df.isna().mean()

In [ ]:
#graphically see the missing data
sns.heatmap(df.isna(), cbar=False)

In [ ]:
df.groupby('title_year')['gross'].mean()

#### Dropping missing rows

One way to handle missing data is just to drop the observation from the data set. This is not always the ideal way since you will lose obseervations, but it might be unavoidable.  For example, we want to predict the gross earnings for each film, so we have to remove those that don't have value for gross.

In [ ]:
df.dropna(subset=['gross'], inplace=True)

In [ ]:
df.shape

In [ ]:
sns.heatmap(df.isnull(), cbar=False)

In [ ]:
#look at all the observations with at least one missing data point
df[df['budget'].isna()].head()

Quite a few films are still missing the values for budget. We do not want to drop this column because we believe it is an important variable, but we must have a value for each observation in order to use it.

**Talk with a partner to think of different ways you can fill in the missing budget values?**

In [ ]:
#you can fill the missing values with the average value of the observations
df['budget'].fillna(df['budget'].mean(), inplace=False)

Another way to fill the missing data

In [ ]:
df.groupby('rating')['gross'].mean().plot(kind='bar')

In [ ]:
budget_ratings = df.groupby('rating')['budget'].mean().round(1).to_dict()
budget_ratings

In [ ]:
df['budget'].fillna(df['rating'].map(budget_ratings), inplace=True)


In [ ]:
sns.heatmap(df.isnull(), cbar=False)

What statistical test could we use to support our use of this method?

### Handling Categorical Data

https://towardsdatascience.com/the-dummys-guide-to-creating-dummy-variables-f21faddb1d40

In [ ]:
df['rating'].value_counts()

In [ ]:
df['rating'].head(10)

In [ ]:
pd.get_dummies(df['rating']).head(10)

In [ ]:
df = pd.concat([df, pd.get_dummies(df['rating'])], 1)
df.head(10)

## Removing Outliers

https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba

In [ ]:
df.boxplot(['gross'])

In [ ]:
above_3_std = df.gross.mean()+(3*df.gross.std())
above_3_std

In [ ]:
df.sort_values('gross', ascending=False)

In [ ]:
# Calculate gross amount that is 3 times above the standard deviation
above_3std = df.gross.mean()+(3*df.gross.std())

### Use a conditional selection to only return values lower than 3 standard deviations above the mean

## Creating New columns based on other columns

In [ ]:
df['actor_1_facebook_likes'].describe()

In [ ]:
df.columns

In [ ]:
df['total_likes'] = df.actor_1_facebook_likes + df.actor_2_facebook_likes + df.actor_3_facebook_likes

### Using np.where

In [ ]:
df[['actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes']].describe()

In [ ]:
# Create a new column called df.superstar where the value is 1
# if df.actor_1_facebook_likes is greater th000 and 0 if not
df['superstar'] = np.where(df['actor_1_facebook_likes']>=30000, 1, 0)

df[['movie_title', 'actor_1_name','actor_1_facebook_likes', 'superstar']].head(10)

### Using df.apply()

In [ ]:
df.head()

In [ ]:
df.apply(lambda row: 1 if row.total_likes> 50000 else 0, axis=1)

In [ ]:
x = 1
y = 2
z = 3

In [ ]:
x==2 or y==3

In [ ]:
# create a function to apply to each row
# function will 
def superstar_movie(row):
    return row.actor_1_facebook_likes> 50000 or (row.actor_2_facebook_likes + row.actor_3_facebook_likes)>25000

In [ ]:
df.apply(lambda row: superstar_movie(row), axis=1).sum()

**Create your own new column of data using the method above.**

In [ ]:
#your code here


Another data cleaning Resource:

https://medium.com/@rrfd/cleaning-and-prepping-data-with-python-for-data-science-best-practices-and-helpful-packages-af1edfbe2a3

## Deep Copy vs. Shallow Copy

In [14]:

#make a copy of the dataframe. 
testing = df
testing.head(2)


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009-01-01,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007-01-01,5000.0,7.1,2.35,0


In [15]:
#change the values for one column
testing['Director'] = 'SeanAbu'
testing.head(2)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,Director
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009-01-01,936.0,7.9,1.78,33000,SeanAbu
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007-01-01,5000.0,7.1,2.35,0,SeanAbu


***Let's look back at the origianl dataframe***


In [16]:
df.head(2)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,Director
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009-01-01,936.0,7.9,1.78,33000,SeanAbu
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007-01-01,5000.0,7.1,2.35,0,SeanAbu




We created a shallow copy of the dataframe.  What does that mean?


In [17]:
# Note you can run .copy on a df and a series
s = pd.Series([1, 2], index=["a", "b"])
deep = s.copy(deep=True)
shallow = s.copy(deep=False)


Shallow copy shares data and index with original.

In [18]:
s is shallow

False

In [19]:
s.values is shallow.values and s.index is shallow.index


True

Deep copy has own copy of data and index.

In [20]:
s is deep

False

In [21]:
s.values is deep.values or s.index is deep.index

False

Updates to the data shared by shallow copy and original is reflected in both; deep copy remains unchanged.

In [22]:
s[0] = 3
shallow[1] = 4

In [23]:
print(s, shallow, deep)



a    3
b    4
dtype: int64 a    3
b    4
dtype: int64 a    1
b    2
dtype: int64


In [24]:
print(shallow)


a    3
b    4
dtype: int64


In [25]:
print( deep)


a    1
b    2
dtype: int64
